In [1]:
import os
import random
import string
import re
import logging
import time
import tensorflow_datasets as tfds
import tensorflow as tf
import tensorflow_text
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import matplotlib.pyplot as plt

/Users/alejandronogales/anaconda3/envs/traductor/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-08-06 22:34:56.738483: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def load_data(path):
    input_file = os.path.join(path)
    with open(input_file, "r", encoding = 'utf8') as f:
        data = f.read()
    return data.split('\n')

spanish_sentences = load_data('data/input/spanish_vocab.txt')
kichwa_sentences = load_data('data/input/kichwa_vocab.txt')

In [3]:
pares_oraciones = []
for index, item in enumerate(spanish_sentences, start=0):
    spanish = item
    kichwa = "[start] " + kichwa_sentences[index] + " [end]"
    pares_oraciones.append((spanish, kichwa))
   

In [4]:
random.shuffle(pares_oraciones)
num_pares_validar = int(0.1 * len(pares_oraciones))
num_pares_train = int(0.9 * len(pares_oraciones))
pares_train = pares_oraciones[:num_pares_train]
pares_validar = pares_oraciones[num_pares_train:num_pares_train + num_pares_validar]

In [5]:
modificar_chars = string.punctuation + "¿"
modificar_chars = modificar_chars.replace("[", "")
modificar_chars = modificar_chars.replace("]", "")

len_sentences = 7
num_vocab = 5000

def get_standardization(text):
    texto = tf.strings.lower(text)
    return tf.strings.regex_replace(
        texto, f"[{re.escape(modificar_chars)}]", "")

texto_source_vectorizado = layers.TextVectorization(
    output_mode = "int",
    output_sequence_length = len_sentences,
    max_tokens = num_vocab,
    standardize = get_standardization
)
texto_target_vectorizado = layers.TextVectorization(
    output_mode = "int",
    output_sequence_length = len_sentences + 1,
    max_tokens = num_vocab,
    standardize = get_standardization
)

textos_train_spanish = [par[0] for par in pares_train]
textos_train_kichwa = [par[1] for par in pares_train]
texto_source_vectorizado.adapt(textos_train_spanish)
texto_target_vectorizado.adapt(textos_train_kichwa)

In [6]:
batch_size = 64

def set_format(spanish, kichwa):
    spanish = texto_source_vectorizado(spanish) 
    kichwa = texto_target_vectorizado(kichwa)
    return (
        {
            "spanish": spanish,
            "kichwa": kichwa[:, :-1],
        }, 
        kichwa[:, 1:]
    )

def make_dataset(pairs):
    textos_spanish, textos_kichwa = zip(*pairs)
    textos_spanish = list(textos_spanish)
    textos_kichwa = list(textos_kichwa)

    dataset = tf.data.Dataset.from_tensor_slices((textos_spanish, textos_kichwa))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(set_format, num_parallel_calls=4)

    return dataset.shuffle(2048).prefetch(16).cache()

dataset_train = make_dataset(pares_train)
dataset_validacion = make_dataset(pares_validar)

for entradas, salidas in dataset_train.take(1):
    print(f"entradas['spanish'].shape: {entradas['spanish'].shape}")
    print(f"entradas['kichwa'].shape: {entradas['kichwa'].shape}")
    print(f"salidas.shape: {salidas.shape}")

entradas['spanish'].shape: (64, 7)
entradas['kichwa'].shape: (64, 7)
salidas.shape: (64, 7)


2023-08-06 22:35:06.870005: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [7]:
class EncoderTrans(layers.Layer):

    def __init__(self, num_heads, dense_dim, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.num_heads = num_heads
        self.dense_dim = dense_dim
        self.embed_dim = embed_dim
        self.attention = layers.MultiHeadAttention(num_heads = num_heads, key_dim = embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(
                dense_dim, 
                activation = "relu"
            ),
            layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, entradas, mask = None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_salida = self.attention(entradas, entradas, attention_mask = mask)
        proj_entrada = self.layernorm_1(entradas + attention_salida)
        proj_salida = self.dense_proj(proj_entrada)

        return self.layernorm_2(proj_entrada + proj_salida)

    def get_config(self):
        config = super().get_config()
        config.update({
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
            "embed_dim": self.embed_dim
        })
        return config

In [8]:
class DecoderTrans(layers.Layer):
    def __init__(self, num_heads, dense_dim, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.num_heads = num_heads
        self.dense_dim = dense_dim
        self.embed_dim = embed_dim
        self.attention_1 = layers.MultiHeadAttention(
            num_heads = num_heads, key_dim = embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads = num_heads, key_dim = embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation = "relu"),
             layers.Dense(embed_dim)]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
            "embed_dim": self.embed_dim
        })
        return config

    def get_mask_attention_causal(self, entradas):
        shape_entrada = tf.shape(entradas)
        batch_size, len_sequence = shape_entrada[0], shape_entrada[1]
        i = tf.range(len_sequence)[:, tf.newaxis]
        j = tf.range(len_sequence)
        mask = tf.cast(i >= j, dtype = "int32")
        mask = tf.reshape(mask, (1, shape_entrada[1], shape_entrada[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype = tf.int32)], axis = 0)
        return tf.tile(mask, mult)

    def call(self, entradas, salidas_encoder, mask = None):
        mask_causal = self.get_mask_attention_causal(entradas)
        if mask is not None:
            mask_padding = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            mask_padding = tf.minimum(mask_padding, mask_causal)
        attention_salida_1 = self.attention_1(
            query = entradas,
            value = entradas,
            key = entradas,
            attention_mask = mask_causal)
        attention_salida_1 = self.layernorm_1(entradas + attention_salida_1)
        attention_salida_2 = self.attention_2(
            query = attention_salida_1,
            value = salidas_encoder,
            key = salidas_encoder,
            attention_mask = mask_padding,
        )
        attention_salida_2 = self.layernorm_2(
            attention_salida_1 + attention_salida_2)
        proj_salida = self.dense_proj(attention_salida_2)

        return self.layernorm_3(attention_salida_2 + proj_salida)

In [9]:
class PositionalEncoding(layers.Layer):

    def __init__(self, dim_entrada, dim_salida, len_sentences, **kwargs):
        super().__init__(**kwargs)
        self.input_dim = dim_entrada
        self.output_dim = dim_salida
        self.token_embeddings = layers.Embedding(
            input_dim = dim_entrada, output_dim = dim_salida)
        self.position_embeddings = layers.Embedding(
            input_dim = len_sentences, output_dim = dim_salida)
        self.sequence_length = len_sentences

    def call(self, entradas):
        limite = tf.shape(entradas)[-1]
        posiciones = tf.range(start = 0, limit = limite, delta = 1)
        tokens_embedded = self.token_embeddings(entradas)
        posiciones_embedded = self.position_embeddings(posiciones)
        return tokens_embedded + posiciones_embedded

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update({
            "input_dim": self.input_dim,
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length
        })
        return config

In [10]:
num_heads = 8
embed_dim = 256
dense_dim = 2048

entradas_encoder = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = PositionalEncoding(num_vocab, embed_dim, len_sentences)(entradas_encoder)
salidas_encoder = EncoderTrans(num_heads, dense_dim, embed_dim )(x)

entradas_decoder = keras.Input(shape=(None,), dtype="int64", name="kichwa")
x = PositionalEncoding(num_vocab, embed_dim, len_sentences,)(entradas_decoder)
x = DecoderTrans(num_heads, dense_dim, embed_dim )(x, salidas_encoder)
x = layers.Dropout(0.5)(x)

salidas_decoder = layers.Dense(num_vocab, activation="softmax")(x)
transformer = keras.Model([entradas_encoder, entradas_decoder], salidas_decoder)

In [ ]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
transformer.summary()
historial = transformer.fit(dataset_train, epochs = 200, validation_data = dataset_validacion)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 spanish (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 kichwa (InputLayer)         [(None, None)]               0         []                            
                                                                                                  
 positional_encoding (Posit  (None, None, 256)            1281792   ['spanish[0][0]']             
 ionalEncoding)                                                                                   
                                                                                                  
 positional_encoding_1 (Pos  (None, None, 256)            1281792   ['kichwa[0][0]']          

52/52 [==============================] - 37s 707ms/step - loss: 0.0372 - accuracy: 0.9936 - val_loss: 3.7965 - val_accuracy: 0.5809
Epoch 40/200
52/52 [==============================] - 37s 710ms/step - loss: 0.0318 - accuracy: 0.9948 - val_loss: 3.8811 - val_accuracy: 0.5773
Epoch 41/200
52/52 [==============================] - 37s 716ms/step - loss: 0.0371 - accuracy: 0.9930 - val_loss: 4.1630 - val_accuracy: 0.4928
Epoch 42/200
52/52 [==============================] - 37s 713ms/step - loss: 0.0645 - accuracy: 0.9891 - val_loss: 3.9140 - val_accuracy: 0.5761
Epoch 43/200
52/52 [==============================] - 36s 700ms/step - loss: 0.0336 - accuracy: 0.9945 - val_loss: 3.8748 - val_accuracy: 0.5791
Epoch 44/200
52/52 [==============================] - 37s 710ms/step - loss: 0.0435 - accuracy: 0.9921 - val_loss: 3.9618 - val_accuracy: 0.5755
Epoch 45/200
52/52 [==============================] - 37s 707ms/step - loss: 0.0331 - accuracy: 0.9935 - val_loss: 3.9630 - val_accuracy: 0.568

52/52 [==============================] - 35s 672ms/step - loss: 0.0158 - accuracy: 0.9962 - val_loss: 4.3014 - val_accuracy: 0.5857
Epoch 96/200
52/52 [==============================] - 36s 695ms/step - loss: 0.0116 - accuracy: 0.9971 - val_loss: 4.3383 - val_accuracy: 0.5839
Epoch 97/200
52/52 [==============================] - 36s 685ms/step - loss: 0.0130 - accuracy: 0.9971 - val_loss: 4.3617 - val_accuracy: 0.5809
Epoch 98/200
52/52 [==============================] - 36s 689ms/step - loss: 0.0124 - accuracy: 0.9967 - val_loss: 4.3617 - val_accuracy: 0.5797
Epoch 99/200
52/52 [==============================] - 35s 673ms/step - loss: 0.0130 - accuracy: 0.9969 - val_loss: 4.4104 - val_accuracy: 0.5827
Epoch 100/200
52/52 [==============================] - 36s 684ms/step - loss: 0.0145 - accuracy: 0.9966 - val_loss: 4.4088 - val_accuracy: 0.5785
Epoch 101/200
52/52 [==============================] - 36s 685ms/step - loss: 0.0193 - accuracy: 0.9957 - val_loss: 4.4064 - val_accuracy: 0.5

In [ ]:
# save the model
transformer.save('output/model_spanish_kichwa')

# save the pares train to use on the server
with open('output/pares_spanish_kichwa.txt', 'w') as f:
    for par in pares_train:
        par_string = ''.join(par)
        f.write(par_string)
        f.write('\n')
print("Saved model to disk")

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('epoch')
plt.ylabel('perdida')
plt.plot(historial.history['loss'])

In [ ]:
vocab_kichwa = texto_target_vectorizado.get_vocabulary()
busqueda_kichwa = dict(zip(range(len(vocab_kichwa)), vocab_kichwa))
max_len_sentence = 7


def traducir(input_sentence):
    tokenized_input_sentence = texto_source_vectorizado([input_sentence])
    sentence_decoded = "[start]"
    for i in range(max_len_sentence):
        sentence_target_tokenized = texto_target_vectorizado(
            [sentence_decoded])[:, :-1]
        predicciones = transformer(
            [tokenized_input_sentence, sentence_target_tokenized])
        index_token = np.argmax(predicciones[0, i, :])
        token = busqueda_kichwa[index_token]
        sentence_decoded += " " + token
        if token == "[end]":
            break
    return sentence_decoded

test_spanish_texts = [pair[0] for pair in pares_train]

for _ in range(10):
    input_sentence = random.choice(test_spanish_texts)
    print(input_sentence)
    print(traducir(input_sentence))
    print("...")

In [ ]:
print(traducir("hola"))